# Cài đặt môi trường và tải Weights

In [ ]:
# 1. Cài đặt thư viện
!pip install timm=="0.4.12" transformers=="4.30.2" fairscale=="0.4.4" pycocoevalcap scikit-learn

# 2. Clone Repo
import os
if not os.path.exists('recognize-anything'):
    !git clone https://github.com/xinyu1205/recognize-anything.git
%cd recognize-anything

# 3. Tải Weights (CÓ KIỂM TRA DUNG LƯỢNG)
def smart_download(url, filename, min_size_bytes):
    should_download = True
    if os.path.exists(filename):
        # Kiểm tra kích thước file
        file_size = os.path.getsize(filename)
        if file_size < min_size_bytes:
            print(f"CẢNH BÁO: File '{filename}' bị lỗi (Size: {file_size/1024/1024:.1f} MB). Đang xóa để tải lại...")
            os.remove(filename)
        else:
            print(f"File '{filename}' đã có sẵn và hợp lệ ({file_size/1024/1024:.1f} MB).")
            should_download = False
            
    if should_download:
        print(f"Đang tải {filename}...")
        !wget {url} -O {filename}
        
        # Kiểm tra lại lần nữa sau khi tải
        if os.path.exists(filename) and os.path.getsize(filename) > min_size_bytes:
            print("🎉 Tải thành công!")
        else:
            print("❌ Tải thất bại! Vui lòng kiểm tra mạng.")

# RAM Large
smart_download(
    "https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth",
    "ram_swin_large_14m.pth",
    1_500_000_000 
)

# Tag2Text
smart_download(
    "https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/tag2text_swin_14m.pth",
    "tag2text_swin_14m.pth",
    800_000_000
)

print("Setup Hoàn Tất!")

# Customize lại bert_config và tag2text

In [ ]:
import os
import json

# fix vocab bert config lai5 chuan63 30524
bert_config_path = 'ram/models/bert_config.json'
config_content = {
  "architectures": ["BertModel"], "attention_probs_dropout_prob": 0.1, "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1, "hidden_size": 768, "initializer_range": 0.02,
  "intermediate_size": 3072, "layer_norm_eps": 1e-12, "max_position_embeddings": 512,
  "model_type": "bert", "num_attention_heads": 12, "num_hidden_layers": 12,
  "pad_token_id": 0, "type_vocab_size": 2, "vocab_size": 30524, "fusion_layer": 0, "encoder_width": 768
}
os.makedirs(os.path.dirname(bert_config_path), exist_ok=True)
with open(bert_config_path, 'w') as f:
    json.dump(config_content, f)
print("Đã vá file bert_config.json")

# fix import của tag2text cho phù hợp với kaggle
tag2text_code = """
import torch
from torch import nn
import torch.nn.functional as F
from .swin_transformer import SwinTransformer
from .bert import BertConfig, BertLMHeadModel, BertModel
import numpy as np
import os
from transformers import BertTokenizer

def build_tokenizer(text_encoder_type):
    tokenizer = BertTokenizer.from_pretrained(text_encoder_type)
    tokenizer.add_special_tokens({'bos_token': '[DEC]', 'additional_special_tokens': ['[ENC]']})
    tokenizer.enc_token_id = tokenizer.additional_special_tokens_ids[0]
    return tokenizer

def create_vit(vit, image_size, vit_grad_ckpt, vit_ckpt_layer):
    if vit == 'swin_b':
        return SwinTransformer(img_size=image_size, patch_size=4, in_chans=3, embed_dim=128, depths=[2, 2, 18, 2], num_heads=[4, 8, 16, 32], window_size=12), 1024
    elif vit == 'swin_l' or vit == 'large':
        return SwinTransformer(img_size=image_size, patch_size=4, in_chans=3, embed_dim=192, depths=[2, 2, 18, 2], num_heads=[6, 12, 24, 48], window_size=12), 1536
    return None, 0

def load_checkpoint_swin(model, url_or_filename):
    checkpoint = torch.load(url_or_filename, map_location='cpu')
    state_dict = checkpoint.get('model', checkpoint)
    full_dict = model.state_dict()
    for k in list(state_dict.keys()):
        if 'relative_position_bias_table' in k:
            if state_dict[k].size() != full_dict[k].size():
                state_dict[k] = F.interpolate(state_dict[k].permute(1, 0).unsqueeze(0), size=full_dict[k].shape[0], mode='linear', align_corners=False).squeeze(0).permute(1, 0)
    model.load_state_dict(state_dict, strict=False)
    return model, "Loaded"

class Tag2Text(nn.Module):
    def __init__(self, image_size=384, text_encoder_type='bert-base-uncased', vit='base', tag_list='../data/tag_list.txt', **kwargs):
        super().__init__()
        self.tokenizer = build_tokenizer(text_encoder_type)
        self.tag_list = self.load_tag_list(tag_list)
        self.num_class = len(self.tag_list)
        self.visual_encoder, vision_width = create_vit(vit, image_size, False, 0)
        med_config = BertConfig.from_json_file(f'{os.path.dirname(os.path.abspath(__file__))}/bert_config.json')
        med_config.encoder_width = vision_width
        self.text_decoder = BertLMHeadModel(config=med_config)
        self.tagging_head = nn.Linear(vision_width, self.num_class)
        self.label_embed = nn.Embedding(self.num_class, 512)
        
    def load_tag_list(self, tag_list_file):
        with open(tag_list_file, 'r') as f: return np.array(f.read().splitlines())

    def forward(self, image, caption=None):
        image_embeds = self.visual_encoder(image)
        return self.tagging_head(image_embeds.mean(dim=1))

def tag2text(pretrained='', **kwargs):
    model = Tag2Text(**kwargs)
    if pretrained: load_checkpoint_swin(model, pretrained)
    return model
"""
with open('ram/models/tag2text.py', 'w') as f:
    f.write(tag2text_code)
print("Đã vá file ram/models/tag2text.py")

# XỬ LÝ DỮ LIỆU (CHIA TRAIN/TEST, TẠO CAPTION và TAG LIST)

In [ ]:
import os
import json
import glob
import random
from sklearn.model_selection import train_test_split


# PHẦN 1: XỬ LÝ DỮ LIỆU & TẠO FILE JSON

# Config
BASE_DIR = '/kaggle/input' 
TRAIN_JSON = 'garbage_train.json'
TEST_JSON = 'garbage_test.json'
T2T_JSON = 'tag2text_train.json'
TAG_FILE = 'garbage_tags.txt'
CLASSES = ['glass', 'paper', 'cardboard', 'plastic', 'metal', 'battery', 'trash']

print("🔄 Đang xử lý dữ liệu...")

# 1. Tạo file tag list
with open(TAG_FILE, 'w') as f:
    for c in CLASSES: f.write(f"{c}\n")

# 2. Quét ảnh từ thư mục Input
label_map = {
    'glass': ['brown-glass', 'green-glass', 'white-glass', 'glass'],
    'paper': ['paper'], 'cardboard': ['cardboard'], 'plastic': ['plastic'],
    'metal': ['metal'], 'battery': ['battery'],
    'trash': ['trash', 'biological', 'shoes', 'clothes']
}
folder_to_label = {src: target for target, srcs in label_map.items() for src in srcs}

all_data = []
for root, dirs, files in os.walk(BASE_DIR):
    for d in dirs:
        if d.lower() in folder_to_label:
            lbl = folder_to_label[d.lower()]
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG']:
                for p in glob.glob(os.path.join(root, d, ext)):
                    all_data.append({"image_path": p, "labels": [lbl]})

if len(all_data) > 0:
    # 3. Chia Train/Test (80/20)
    random.shuffle(all_data)
    train, test = train_test_split(all_data, test_size=0.2, random_state=42)
    
    with open(TRAIN_JSON, 'w') as f: json.dump(train, f, indent=4)
    with open(TEST_JSON, 'w') as f: json.dump(test, f, indent=4)
    
    # 4. Tạo dữ liệu cho Tag2Text (Sinh caption giả)
    templates = ["A photo of {}.", "This is {}.", "Image of {}."]
    t2t_data = [{"image_path": i['image_path'], "tag": i['labels'], "caption": random.choice(templates).format(i['labels'][0])} for i in train]
    with open(T2T_JSON, 'w') as f: json.dump(t2t_data, f, indent=4)
    
    print(f"✅ Xử lý dữ liệu xong: Train ({len(train)}) | Test ({len(test)})")
else:
    print("❌ LỖI: Không tìm thấy ảnh! Hãy kiểm tra lại đường dẫn Input.")


# PHẦN 2: TẠO FILE CODE 'custom_datasets.py'

dataset_code_content = """
import torch
from torch.utils.data import Dataset
from PIL import Image
import json
from transformers import BertTokenizer

CLASSES = ['glass', 'paper', 'cardboard', 'plastic', 'metal', 'battery', 'trash']

class GarbageDataset(Dataset):
    def __init__(self, json_file, transform=None):
        with open(json_file, 'r') as f: self.data = json.load(f)
        self.transform = transform
        self.classes = CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        try: image = Image.open(item['image_path']).convert('RGB')
        except: image = Image.new('RGB', (384, 384))
        
        if self.transform: image = self.transform(image)
        
        label_tensor = torch.zeros(len(self.classes))
        # Hỗ trợ cả key 'labels' (RAM) và 'tag' (Tag2Text)
        labels = item.get('labels', item.get('tag', []))
        
        for name in labels:
            if name in self.class_to_idx:
                label_tensor[self.class_to_idx[name]] = 1.0
        return image, label_tensor

class Tag2TextDataset(Dataset):
    def __init__(self, json_file, transform=None):
        with open(json_file, 'r') as f: self.data = json.load(f)
        self.transform = transform
        self.classes = CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        try: image = Image.open(item['image_path']).convert('RGB')
        except: image = Image.new('RGB', (384, 384))
        
        if self.transform: image = self.transform(image)
        
        tag_tensor = torch.zeros(len(self.classes))
        tags = item.get('tag', item.get('labels', []))
        
        for t in tags:
            if t in self.class_to_idx: tag_tensor[self.class_to_idx[t]] = 1.0
            
        caption = item.get('caption', "")
        encoding = self.tokenizer(caption, padding='max_length', truncation=True, max_length=30, return_tensors="pt")
        return image, tag_tensor, encoding.input_ids.squeeze(), encoding.attention_mask.squeeze()
"""

# Ghi nội dung vào file
with open('custom_datasets.py', 'w') as f:
    f.write(dataset_code_content)

print("Đã tạo xong file code: custom_datasets.py")

# ĐỊNH NGHĨA DATASET CLASSES

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from transformers import BertTokenizer
import json

class GarbageDataset(Dataset):
    def __init__(self, json_file, transform=None):
        with open(json_file) as f: self.data = json.load(f)
        self.transform = transform
        self.cls_map = {c: i for i, c in enumerate(CLASSES)}
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        try: img = Image.open(item['image_path']).convert('RGB')
        except: img = Image.new('RGB', (384, 384))
        if self.transform: img = self.transform(img)
        lbl = torch.zeros(len(CLASSES)); lbl[self.cls_map[item['labels'][0]]] = 1.0
        return img, lbl

class Tag2TextDataset(Dataset):
    def __init__(self, json_file, transform=None):
        with open(json_file) as f: self.data = json.load(f)
        self.transform = transform
        self.cls_map = {c: i for i, c in enumerate(CLASSES)}
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        try: img = Image.open(item['image_path']).convert('RGB')
        except: img = Image.new('RGB', (384, 384))
        if self.transform: img = self.transform(img)
        tag = torch.zeros(len(CLASSES)); tag[self.cls_map[item['tag'][0]]] = 1.0
        enc = self.tokenizer(item['caption'], padding='max_length', truncation=True, max_length=30, return_tensors="pt")
        return img, tag, enc.input_ids.squeeze(), enc.attention_mask.squeeze()

# TRAINING RAM (SWIM LARGE)

In [ ]:
%%writefile train_ram.py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from ram.models import ram
from custom_datasets import GarbageDataset
import numpy as np
from torch.cuda.amp import autocast, GradScaler
import gc

# --- CẤU HÌNH ---
PRETRAINED = 'ram_swin_large_14m.pth'
TRAIN_FILE = 'garbage_train.json'
# Batch size nhỏ (4) giúp CutMix hoạt động hiệu quả trên P100 mà không tràn RAM
BATCH_SIZE = 4
LR = 1e-5
EPOCHS = 1

# --- HÀM TẠO HỘP CẮT (BOUNDING BOX) ---
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    # Tính kích thước vùng cắt dựa trên tỷ lệ lambda
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # Chọn tâm ngẫu nhiên
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # Tính toạ độ 4 góc
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def main():
    gc.collect(); torch.cuda.empty_cache()
    device = torch.device('cuda')
    
    # Transform chuẩn
    transform = transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    ds = GarbageDataset(TRAIN_FILE, transform)
    dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

    print("🚀 Training RAM với chiến thuật CutMix (Mosaic)...")
    
    model = ram(pretrained=PRETRAINED, image_size=384, vit='swin_l')
    model.tagging_head = nn.Linear(1536, 7)
    model = model.to(device)
    
    # Mở khóa toàn bộ để học features cục bộ
    for p in model.parameters(): p.requires_grad = True
    
    optim = torch.optim.AdamW(model.parameters(), lr=LR)
    crit = nn.BCEWithLogitsLoss()
    scaler = GradScaler()

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for i, (img, tgt) in enumerate(dl):
            img, tgt = img.to(device), tgt.to(device)
            optim.zero_grad()
            
            with autocast():
                # --- THUẬT TOÁN CUTMIX ---
                # Áp dụng 50% số lần (để model vẫn học được ảnh gốc)
                if np.random.rand() < 0.5:
                    # 1. Tạo lambda (tỷ lệ diện tích)
                    lam = np.random.beta(1.0, 1.0)
                    
                    # 2. Hoán đổi vị trí ảnh trong batch
                    rand_index = torch.randperm(img.size()[0]).cuda()
                    target_a = tgt
                    target_b = tgt[rand_index]
                    
                    # 3. Tạo vùng cắt
                    bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                    
                    # 4. Dán ảnh B đè lên vùng cắt của ảnh A
                    img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                    
                    # 5. Tính lại tỷ lệ lambda chính xác theo pixel
                    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                    
                    # 6. Forward & Tính Loss hỗn hợp
                    embeds = model.visual_encoder(img)
                    if len(embeds.shape)==3: embeds = embeds.mean(dim=1)
                    output = model.tagging_head(embeds)
                    loss = crit(output, target_a) * lam + crit(output, target_b) * (1. - lam)
                    
                else:
                    # Train ảnh thường
                    embeds = model.visual_encoder(img)
                    if len(embeds.shape)==3: embeds = embeds.mean(dim=1)
                    loss = crit(model.tagging_head(embeds), tgt)
            
            scaler.scale(loss).backward(); scaler.step(optim); scaler.update()
            total_loss += loss.item()
            
            if i%100==0: print(f"Ep {epoch+1} Step {i} Loss: {loss.item():.4f}")
        
        print(f"=== Epoch {epoch+1} Avg Loss: {total_loss/len(dl):.4f} ===")
        torch.save(model.state_dict(), "ram_best.pth")

if __name__ == '__main__': main()

# TRAINING TAG2TEXT (SWIM BASE)

In [ ]:
%%writefile train_tag2text.py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig, BertLMHeadModel
import os
import numpy as np
# Import Swin từ thư mục repo
try: from ram.models.swin_transformer import SwinTransformer
except: import sys; sys.path.append('recognize-anything'); from ram.models.swin_transformer import SwinTransformer

# --- CẤU HÌNH ---
PRETRAINED = 'tag2text_swin_14m.pth'
JSON_FILE = 'tag2text_train.json'
TAG_FILE = 'garbage_tags.txt'
BATCH_SIZE = 4
LR = 1e-5
EPOCHS = 1

# --- 1. HÀM LOAD CHECKPOINT THÔNG MINH (BỎ QUA LỖI SIZE) ---
def load_smart(model, path):
    print(f"🔄 Đang nạp weights từ {path} (Chế độ bỏ qua lỗi kích thước)...")
    ckpt = torch.load(path, map_location='cpu')
    state = ckpt.get('model', ckpt)
    model_dict = model.state_dict()
    
    # Lọc bỏ key lệch size (Ví dụ: label_embed cũ 3429 -> mới 7)
    new_state = {}
    for k, v in state.items():
        if k in model_dict:
            if v.shape == model_dict[k].shape:
                new_state[k] = v
            # Fix Swin bias (Nội suy kích thước)
            elif 'relative_position_bias_table' in k:
                if v.shape != model_dict[k].shape:
                    new_state[k] = F.interpolate(v.permute(1,0).unsqueeze(0), size=model_dict[k].shape[0], mode='linear', align_corners=False).squeeze(0).permute(1,0)
    
    # Load vào model (strict=False để không báo lỗi thiếu key)
    msg = model.load_state_dict(new_state, strict=False)
    print("✅ Đã nạp xong weights! (Các key lệch size đã bị loại bỏ)")

# --- 2. CLASS MODEL TỰ ĐỊNH NGHĨA (KHÔNG PHỤ THUỘC FILE .PY CŨ) ---
class Tag2Text_Custom(nn.Module):
    def __init__(self, tag_list_file):
        super().__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenizer.add_special_tokens({'bos_token': '[DEC]', 'additional_special_tokens': ['[ENC]']})
        self.tokenizer.enc_token_id = self.tokenizer.additional_special_tokens_ids[0]
        
        # Load Tag List
        with open(tag_list_file, 'r') as f:
            self.tag_list = np.array(f.read().splitlines())
        self.num_class = len(self.tag_list)
        
        # Vision: Swin Base (128 dim) - Khớp với checkpoint
        self.visual_encoder = SwinTransformer(img_size=384, patch_size=4, in_chans=3, embed_dim=128, depths=[2, 2, 18, 2], num_heads=[4, 8, 16, 32], window_size=12)
        
        # Text: BERT (Config cứng)
        # Tự tạo config nếu file không tồn tại hoặc lỗi
        bert_cfg = BertConfig(vocab_size=30524, hidden_size=768, num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072, encoder_width=1024)
        self.text_decoder = BertLMHeadModel(config=bert_cfg)
        
        # Heads mới (7 class)
        self.tagging_head = nn.Linear(1024, self.num_class)
        self.label_embed = nn.Embedding(self.num_class, 512)

    def forward(self, img, input_ids=None):
        emb = self.visual_encoder(img)
        emb_pool = emb.mean(dim=1)
        tag_logits = self.tagging_head(emb_pool)
        
        loss_text = 0
        if input_ids is not None:
            # Truyền nhãn vào BERT để sinh caption
            loss_text = self.text_decoder(input_ids, encoder_hidden_states=emb, labels=input_ids).loss
        return tag_logits, loss_text

# --- 3. MAIN TRAINING ---
def main():
    # Import Dataset tại đây để tránh lỗi import vòng
    from custom_datasets import Tag2TextDataset
    
    device = torch.device('cuda')
    print("🚀 Training Tag2Text (Custom Version)...")
    
    # Dataset
    trans = transforms.Compose([transforms.Resize((384, 384)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    ds = Tag2TextDataset(JSON_FILE, trans)
    dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    
    # Model Init & Load
    model = Tag2Text_Custom(tag_list_file=TAG_FILE)
    load_smart(model, PRETRAINED) # Dùng hàm load thông minh
    
    model = model.to(device)
    optim = torch.optim.AdamW(model.parameters(), lr=LR)
    crit = nn.BCEWithLogitsLoss()
    scaler = GradScaler()
    
    # Loop
    for ep in range(EPOCHS):
        model.train()
        total_loss = 0
        for i, (img, tag, txt, _) in enumerate(dl):
            img, tag, txt = img.to(device), tag.to(device), txt.to(device)
            optim.zero_grad()
            with autocast():
                tag_logits, loss_text = model(img, txt)
                loss = crit(tag_logits, tag) + loss_text
            
            scaler.scale(loss).backward(); scaler.step(optim); scaler.update()
            total_loss += loss.item()
            
            if i%50==0: print(f"Ep {ep+1} Step {i} Loss: {loss.item():.4f}")
        
        print(f"=== Epoch {ep+1} Done. Avg Loss: {total_loss/len(dl):.4f} ===")
        torch.save(model.state_dict(), "tag2text_finetuned.pth")
    
    print("Done Training Tag2Text!")

if __name__ == '__main__': main()

# Chạy Training

In [ ]:
# Chạy Train RAM
!python train_ram.py

# Chạy Train Tag2Text
!python train_tag2text.py

# DEMO & BENCHMARK (RAM VS TAG2TEXT)

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
from ram.models import ram
import torch.nn as nn
import os
import json
import random
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertConfig, BertLMHeadModel
# Import Swin
try: from ram.models.swin_transformer import SwinTransformer
except: import sys; sys.path.append('recognize-anything'); from ram.models.swin_transformer import SwinTransformer


# Đường dẫn của file model đã input
PATH_RAM = "/kaggle/input/regconize-anything-model-finetuned/recognize-anything/ram_best.pth" 
PATH_T2T = "/kaggle/input/regconize-anything-model-finetuned/recognize-anything/tag2text_finetuned.pth"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CLASSES = ['glass', 'paper', 'cardboard', 'plastic', 'metal', 'battery', 'trash']
transform = transforms.Compose([transforms.Resize((384, 384)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# --- 1. LOAD RAM ---
print("===Đang nạp RAM...===")
model_ram = ram(pretrained='ram_swin_large_14m.pth', image_size=384, vit='swin_l')
model_ram.tagging_head = nn.Linear(1536, 7)

if os.path.exists(PATH_RAM):
    print(f"===Đã tìm thấy: {PATH_RAM}")
    model_ram.load_state_dict(torch.load(PATH_RAM, map_location=device))
else:
    print(f"XXX CẢNH BÁO: Không thấy file {PATH_RAM}. Đang dùng model rỗng!")

model_ram.to(device).eval()

# --- 2. LOAD TAG2TEXT (Class Inference) ---
print("⏳ Đang nạp Tag2Text...")
class Tag2Text_Inference(nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenizer.padding_side = 'left'
        self.visual_encoder = SwinTransformer(img_size=384, patch_size=4, in_chans=3, embed_dim=128, depths=[2, 2, 18, 2], num_heads=[4, 8, 16, 32], window_size=12)
        bert_config_path = 'ram/models/bert_config.json'
        if not os.path.exists(bert_config_path): bert_config_path = '/kaggle/working/recognize-anything/ram/models/bert_config.json'
        med_config = BertConfig.from_json_file(bert_config_path)
        med_config.encoder_width = 1024
        self.text_decoder = BertLMHeadModel(config=med_config)
        self.tagging_head = nn.Linear(1024, 7)
        self.label_embed = nn.Embedding(7, 512)

    def generate(self, img, tags):
        emb = self.visual_encoder(img)
        prompt = [f"A photo of {t}." for t in tags]
        inp = self.tokenizer(prompt, padding='max_length', truncation=True, max_length=30, return_tensors="pt").to(img.device)
        with torch.no_grad():
            out = self.text_decoder.generate(input_ids=inp.input_ids, attention_mask=inp.attention_mask, encoder_hidden_states=emb, max_new_tokens=20, num_beams=1)
        return self.tokenizer.batch_decode(out, skip_special_tokens=True)

model_t2t = Tag2Text_Inference()
if os.path.exists(PATH_T2T):
    print(f"===Đã tìm thấy: {PATH_T2T}")
    # Load strict=False để an toàn
    model_t2t.load_state_dict(torch.load(PATH_T2T, map_location=device), strict=False)
else:
    print(f"XXX CẢNH BÁO: Không thấy file {PATH_T2T}. Đang dùng model rỗng!")

model_t2t.to(device).eval()

# --- 3. WIDGET DEMO ---
print("🚀 Hệ thống sẵn sàng!")
btn = widgets.Button(description='Demo Nhanh', button_style='primary', layout=widgets.Layout(width='200px'))
out = widgets.Output()

def on_click(b):
    with out:
        clear_output()
        if not os.path.exists('garbage_test.json'): print("Chưa có file test!"); return
        with open('garbage_test.json') as f: data = json.load(f)
        
        # Mix 4 ảnh
        items = random.sample(data, 4)
        mosaic = Image.new('RGB', (384, 384))
        for idx, pos in enumerate([(0,0), (192,0), (0,192), (192,192)]):
            mosaic.paste(Image.open(items[idx]['image_path']).convert('RGB').resize((192, 192)), pos)
        
        plt.figure(figsize=(6,6)); plt.imshow(mosaic); plt.axis('off'); plt.show()
        tens = transform(mosaic).unsqueeze(0).to(device)
        
        # RAM Predict
        with torch.no_grad():
            emb = model_ram.visual_encoder(tens)
            if len(emb.shape)==3: emb=emb.mean(dim=1)
            probs = torch.sigmoid(model_ram.tagging_head(emb))[0]
            # Ngưỡng thấp (5%) để bắt nhiều vật thể
            found = [f"{CLASSES[i].upper()} ({p:.0%})" for i,p in enumerate(probs) if p>0.05]
            
        # T2T Predict
        with torch.no_grad():
            emb_t2t = model_t2t.visual_encoder(tens)
            top_tag = CLASSES[torch.sigmoid(model_t2t.tagging_head(emb_t2t.mean(dim=1)))[0].argmax()]
            cap = model_t2t.generate(tens, [top_tag])[0]

        print(f"📦 Thực tế: {', '.join([i['labels'][0] for i in items])}")
        print("-" * 40)
        print(f"🤖 RAM thấy: {', '.join(found)}")
        print(f"🦜 T2T mô tả: \"{cap}\"")

btn.on_click(on_click)
display(btn, out)

# DEMO (MOSAIC MIX 4 Ảnh)

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import json
import random
import os
import numpy as np
from transformers import BertTokenizer, BertConfig, BertLMHeadModel

# --- HÀM TỰ ĐỘNG TÌM FILE MODEL ---
def find_model_path(filename):
    # 1. Tìm trong thư mục hiện tại (nếu vừa train xong)
    if os.path.exists(filename):
        return filename
    
    # 2. Tìm trong thư mục Input (nếu mở lại session)
    print(f"🔍 Đang tìm file '{filename}' trong Input...")
    for root, dirs, files in os.walk('/kaggle/input'):
        if filename in files:
            full_path = os.path.join(root, filename)
            print(f"   ✅ Đã tìm thấy: {full_path}")
            return full_path
            
    print(f"   ❌ KHÔNG TÌM THẤY '{filename}'. Hãy chắc chắn bạn đã 'Add Input' output của phiên trước.")
    return None

# --- CẤU HÌNH ---
# Tự động tìm đường dẫn
RAM_PATH = find_model_path("ram_best.pth")
T2T_PATH = find_model_path("tag2text_finetuned.pth")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CLASSES = ['glass', 'paper', 'cardboard', 'plastic', 'metal', 'battery', 'trash']
transform = transforms.Compose([transforms.Resize((384, 384)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


# 1. ĐỊNH NGHĨA LẠI MODEL

try: from ram.models.swin_transformer import SwinTransformer
except: import sys; sys.path.append('recognize-anything'); from ram.models.swin_transformer import SwinTransformer
from ram.models import ram

class Tag2Text_Inference(nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenizer.padding_side = 'left'
        self.visual_encoder = SwinTransformer(img_size=384, patch_size=4, in_chans=3, embed_dim=128, depths=[2, 2, 18, 2], num_heads=[4, 8, 16, 32], window_size=12)
        bert_config_path = 'ram/models/bert_config.json'
        if not os.path.exists(bert_config_path): bert_config_path = '/kaggle/working/recognize-anything/ram/models/bert_config.json'
        med_config = BertConfig.from_json_file(bert_config_path)
        med_config.encoder_width = 1024
        self.text_decoder = BertLMHeadModel(config=med_config)
        self.tagging_head = nn.Linear(1024, 7)
        self.label_embed = nn.Embedding(7, 512)

    def generate(self, img, tags):
        emb = self.visual_encoder(img)
        prompt = [f"A photo of {t}." for t in tags]
        inp = self.tokenizer(prompt, padding='max_length', truncation=True, max_length=30, return_tensors="pt").to(img.device)
        with torch.no_grad():
            out = self.text_decoder.generate(input_ids=inp.input_ids, attention_mask=inp.attention_mask, encoder_hidden_states=emb, max_new_tokens=20, num_beams=1)
        return self.tokenizer.batch_decode(out, skip_special_tokens=True)


# 2. LOAD WEIGHTS

print("\n⏳ Đang nạp Models...")

# Load RAM
model_ram = ram(pretrained='ram_swin_large_14m.pth', image_size=384, vit='swin_l')
model_ram.tagging_head = nn.Linear(1536, 7)

if RAM_PATH:
    model_ram.load_state_dict(torch.load(RAM_PATH, map_location=device))
    print("✅ RAM: Đã nạp finetuned weights.")
else:
    print("⚠️ RAM: Đang dùng weight gốc (chưa finetune)!")

model_ram.to(device).eval()

# Load Tag2Text
model_t2t = Tag2Text_Inference()

if T2T_PATH:
    model_t2t.load_state_dict(torch.load(T2T_PATH, map_location=device), strict=False)
    print("✅ Tag2Text: Đã nạp finetuned weights.")
else:
    print("⚠️ Tag2Text: Đang dùng weight rỗng!")

model_t2t.to(device).eval()
print("🚀 Hệ thống sẵn sàng!")


# 3. WIDGET DEMO

btn = widgets.Button(description='🎲 Mix & Test', button_style='primary', layout=widgets.Layout(width='200px'))
out = widgets.Output()

def on_click(b):
    with out:
        clear_output()
        if not os.path.exists('garbage_test.json'): print("⚠️ Chưa có file test data! Hãy chạy Cell 2 trước."); return
        with open('garbage_test.json') as f: data = json.load(f)
        
        items = random.sample(data, 4)
        mosaic = Image.new('RGB', (384, 384))
        for idx, pos in enumerate([(0,0), (192,0), (0,192), (192,192)]):
            mosaic.paste(Image.open(items[idx]['image_path']).convert('RGB').resize((192, 192)), pos)
        
        plt.figure(figsize=(6,6)); plt.imshow(mosaic); plt.axis('off'); plt.show()
        tens = transform(mosaic).unsqueeze(0).to(device)
        
        # Predict
        with torch.no_grad():
            # RAM
            emb = model_ram.visual_encoder(tens)
            if len(emb.shape)==3: emb=emb.mean(dim=1)
            probs = torch.sigmoid(model_ram.tagging_head(emb))[0]
            found_ram = [f"{CLASSES[i].upper()} ({p:.0%})" for i,p in enumerate(probs) if p>0.05]
            
            # T2T
            emb_t2t = model_t2t.visual_encoder(tens)
            top_tag = CLASSES[torch.sigmoid(model_t2t.tagging_head(emb_t2t.mean(dim=1)))[0].argmax()]
            cap = model_t2t.generate(tens, [top_tag])[0]

        print(f"📦 Thực tế: {', '.join([i['labels'][0] for i in items])}")
        print("-" * 40)
        print(f"🤖 RAM phát hiện: {', '.join(found_ram)}")
        print(f"📝 Tag2Text mô tả: \"{cap}\"")

btn.on_click(on_click)
display(btn, out)